<center> <h2> Spring 2022 DS 4400 </h2></center>

<center> <h3> Homework 1 </h3></center>

<center> <h3> Author: Yichen Sun </h3></center>

## Outline

1. <a href='#1'>Problem 4 [Matrix transpose and inverse]</a>
2. <a href='#2'>Problem 5 [Average, variance, and correlation]</a>

## 1. Problem 4 [Matrix transpose and inverse] 20 points

c) Generate at random 3 matrices of size 3 × 3 and fill each entry with a random integer chosen from -10 to 10. Use an existing package to compute the inverse of each matrix if it exists. 

Include in your report:

* Each of the 3 matrices
* The inverses computed with the package
* Compute the product of each matrix with its inverse to check that you obtain the identity matrix.

In [1]:
import numpy as np

In [2]:
def generate_matrices():
    
    for i in range(3):
        matrix = np.random.randint(-10,10, size=(3,3))
        print(f'matrix {i}:')
        print(matrix)
        try:
            inverse = np.linalg.inv(matrix)
        except np.linalg.LinAlgError:
            pass
        else:
            print(f'matrix {i} inverse:')
            print(inverse)
            
        product = np.dot(matrix, inverse)
        identity = np.eye(3)
        isIdentity = np.all(np.equal(product, identity))
        if isIdentity:
            print('you obtain the identity matrix')
        else:
            print(f'matrix {i} is not an identity matrix')

In [3]:
generate_matrices()

matrix 0:
[[-7  8 -5]
 [ 9  6  5]
 [ 9 -4 -5]]
matrix 0 inverse:
[[-0.00806452  0.0483871   0.05645161]
 [ 0.07258065  0.06451613 -0.00806452]
 [-0.07258065  0.03548387 -0.09193548]]
matrix 0 is not an identity matrix
matrix 1:
[[ -7  -3   7]
 [  4   0  -7]
 [-10 -10   0]]
matrix 1 inverse:
[[-9.85162418e+14 -9.85162418e+14  2.95548726e+14]
 [ 9.85162418e+14  9.85162418e+14 -2.95548726e+14]
 [-5.62949953e+14 -5.62949953e+14  1.68884986e+14]]
matrix 1 is not an identity matrix
matrix 2:
[[ -2  -2 -10]
 [  7 -10  -9]
 [ -2   6   0]]
matrix 2 inverse:
[[-0.14835165  0.16483516  0.22527473]
 [-0.04945055  0.05494505  0.24175824]
 [-0.06043956 -0.04395604 -0.09340659]]
matrix 2 is not an identity matrix


## 2. Problem 5 [Average, variance, and correlation] 20 points

### Dataset

In [4]:
import pandas as pd
import seaborn as sns

In [5]:
kc_house_data = pd.read_csv (r'/Users/mac/Downloads/kc_house_data.csv')

In [6]:
kc_house_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [7]:
# Ignore the columns id and date, as well as the categorical column zipcode
# One solution is to remove them
df = kc_house_data.drop(['id', 'date', 'zipcode'], axis=1)

In [8]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503


a) For each feature, write code to compute the average value, the min and max values, as well as its variance.

In [9]:
feature_list = [
    'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
    'view', 'condition', 'grade', 'sqft_above', 'sqft_basement',  'yr_built', 'yr_renovated', 
    'lat', 'long', 'sqft_living15', 'sqft_lot15']

def compute_features(df):
    
    mean_dict = {}
    var_dict = {}
    
    for feature in feature_list:
        num_rows = len(df[feature])
        mean = df[feature].mean()
        maximum = df[feature].max()
        minimum = df[feature].min()
        var = sum((x - mean) ** 2 for x in df[feature]) / (num_rows - 1)
        
        mean_dict[feature] = mean
        var_dict[feature] = var
        print(f'{df[feature].name} - mean: {mean}, min: {minimum}, max: {maximum}, variance: {var}')
    
    return mean_dict, var_dict
        
mean, variance = compute_features(df)

price - mean: 540088.1417665294, min: 75000.0, max: 7700000.0, variance: 134782378397.24681
bedrooms - mean: 3.37084162309721, min: 0, max: 33, variance: 0.8650150097573724
bathrooms - mean: 2.1147573219821405, min: 0.0, max: 8.0, variance: 0.5931512887355798
sqft_living - mean: 2079.8997362698374, min: 290, max: 13540, variance: 843533.6813681519
sqft_lot - mean: 15106.967565816869, min: 520, max: 1651359, variance: 1715658774.1754541
floors - mean: 1.4943089807060566, min: 1.0, max: 3.5, variance: 0.29158800687709074
waterfront - mean: 0.007541757275713691, min: 0, max: 1, variance: 0.007485225502689098
view - mean: 0.23430342849211122, min: 0, max: 4, variance: 0.5872426169774596
condition - mean: 3.4094295100171195, min: 1, max: 5, variance: 0.42346651239404876
grade - mean: 7.656873178179799, min: 1, max: 13, variance: 1.3817032893475767
sqft_above - mean: 1788.3906907879516, min: 290, max: 9410, variance: 685734.6672685045
sqft_basement - mean: 291.5090454818859, min: 0, max: 482

Which features have the lowest and the highest average? Include the feature name and their average values for the features with the lowest and highest average.

In [32]:
max_value = max(mean.values())

for i in mean.items():
    if i[1] == max_value:
        print(f'The highest average is {i[0]}: {i[1]}')
        
min_value = min(mean.values())

for i in mean.items():
    if i[1] == min_value:
        print(f'The lowest average is {i[0]}: {i[1]}')

The highest average is price: 540088.1417665294
The lowest average is long: -122.21389640494147


Which features have the lowest and the highest variance? Include the feature name and their variance values for the features with the lowest and highest variance.

In [33]:
max_var = max(variance.values())

for i in variance.items():
    if i[1] == max_var:
        print(f'The highest variance is {i[0]}: {i[1]}')
        
min_var = min(variance.values())

for i in variance.items():
    if i[1] == min_var:
        print(f'The lowest variance is {i[0]}: {i[1]}')

The highest variance is price: 134782378397.24681
The lowest variance is waterfront: 0.007485225502689098


b) Compute the correlation coefficient of each feature with the response. Include a table with the correlation coefficient of each feature with the response. Which features are positively correlated (i.e., have positive correlation coefficient) with the response? Which feature has the highest positive correlation with the response?

In [12]:
features = [
    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
    'view', 'condition', 'grade', 'sqft_above', 'sqft_basement',  'yr_built', 'yr_renovated', 
    'lat', 'long', 'sqft_living15', 'sqft_lot15']

data = {}

def compute_corr():
    
    for feature in features:
        feature_corr = np.corrcoef(df['price'], df[feature])
        
        data[feature] = feature_corr[0][1]
        
        corr_table = pd.DataFrame(data, index=['price'])

    return corr_table
        
df_corr = compute_corr()

In [13]:
df_corr

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
price,0.30835,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,0.307003,0.021626,0.585379,0.082447


In [35]:
highest_corr = max(data.values())

for i in data.items():
    if i[1] == highest_corr:
        print(f'The highest positive correlation with the response is {i[0]}: {i[1]}')


The highest positive correlation with the response is sqft_living: 0.7020350546118003


c) Were you able to find any features with a negative correlation coefficient with the response?

`I am not able to find any features with a negative correlation coefficient with the response.`